In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os

from experiment import *

# Load data ** Chose your path before get stated **

In [2]:
path = os.getcwd()
rootDir, _ = path.split('ECG-Arritmia-Paper1')
path = 'ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features'
filename = 'MIT-BIH__DS2_5classes__Fourier.csv'
dataset = pd.read_csv(os.path.join(rootDir, path, filename));

In [3]:
dataset.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,target
0,0.168067,0.193887,0.216915,0.210637,0.169596,0.123020,0.076533,0.051240,0.034684,0.024381,...,0.001717,0.001402,0.001021,0.001168,0.001496,0.001062,0.001859,0.001419,0.000781,0
1,0.150410,0.277900,0.282133,0.254650,0.182793,0.103832,0.052994,0.025424,0.014123,0.009648,...,0.001103,0.001164,0.000873,0.002985,0.000949,0.002357,0.001331,0.000781,0.003125,0
2,0.052129,0.221438,0.240856,0.228624,0.198214,0.140595,0.099769,0.062602,0.048336,0.030559,...,0.001838,0.001028,0.000912,0.001266,0.001506,0.000536,0.001411,0.001822,0.002344,0
3,0.000996,0.212796,0.247697,0.230758,0.180031,0.126334,0.088010,0.060599,0.043856,0.037445,...,0.000625,0.002388,0.000985,0.000235,0.001016,0.001002,0.000258,0.002047,0.001719,0
4,0.027285,0.189900,0.209953,0.205292,0.178175,0.133576,0.090180,0.057837,0.039776,0.023964,...,0.001762,0.000873,0.001418,0.001064,0.001545,0.000357,0.001771,0.001129,0.000625,0


In [4]:
dataset.shape

(49668, 34)

# Data preprocessing

In this step we are going standardize our dataset. The pipeline.pkl will be loaded, and it contains the weights of normalization onf DS1 set.

In [5]:
from sklearn.externals import joblib

pipeline = joblib.load('./Models/pipeline.pkl')

Separete features from labels.

In [8]:
data_std = pipeline.transform(dataset.values[:,:-1])
data_label = dataset.values[:,-1]

# Load classifiers

In [9]:
from sklearn.externals import joblib

random_search_SVM = joblib.load('./Models/classifiers_SVM.pkl')
random_search_MLP = joblib.load('./Models/classifiers_MLP.pkl')
random_search_Bayes = joblib.load('./Models/classifiers_MLP_Bayes.pkl')

In [10]:
svm_clf = random_search_SVM['SVM-RBF'].best_estimator_
mlp_clf = random_search_MLP['MLP'].best_estimator_
bayes_clf = random_search_Bayes['Naive-Bayes']

In [11]:
classifiers = {'MLP-768': mlp_clf, 'SVM-RBF': svm_clf, 'Naive-Bayes': bayes_clf}

# **TEST** on DS2 set

In [65]:
clf_outputs = {'true': dict((k, {}) for k in classifiers.keys()), 
               'pred': dict((k, {}) for k in classifiers.keys())}
results = dict((k, {}) for k in classifiers.keys())

for clf in classifiers.keys():
    print(clf)
    clf_outputs['true'][clf][0] = data_label
    clf_outputs['pred'][clf][0] = classifiers[clf].predict(data_std)
    

results = results_clf(5, clf_outputs['true'], clf_outputs['pred'])

MLP-768
SVM-RBF
Naive-Bayes


/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Fourier/Experiment_11_scikitLearn__allCLF_onDS2_allFeatures/experiment.py:150: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,2] = TP / (TP + FP)
/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Fourier/Experiment_11_scikitLearn__allCLF_onDS2_allFeatures/experiment.py:152: RuntimeWarning: invalid value encountered in double_scalars
  metrics_

# Save results to CSVs and figures

In [70]:
export_results(results, 'validation')

MLP-768
SVM-RBF
Naive-Bayes


/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Fourier/Experiment_11_scikitLearn__allCLF_onDS2_allFeatures/experiment.py:286: RuntimeWarning: invalid value encountered in true_divide
  return np.around((confMat / np.sum(confMat,axis=1)[:,None])*100,2)
